# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit e2242eca6dd4920c56f271aa93798c912131abaf
Author: Najwa Ez Zine <nezzine@frennes>
Date:   Thu Jun 7 16:11:01 2018 +0200

    Theory ressources


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl


In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 4
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [6]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

A   job is already running, using it. jobid is 1015246


On les range

In [7]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-23.rennes.grid5000.fr'),
 Host('parasilo-24.rennes.grid5000.fr'),
 Host('parasilo-25.rennes.grid5000.fr'),
 Host('parasilo-26.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [80]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [81]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

#### Test : commande simple 'ls'

In [ ]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [82]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [83]:
Remote_install.run().ok

True

#### Version Simgrid

In [84]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [18]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [8]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

In [9]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [10]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        line = l[i].split()
        S[int(line[1])] = int(float(line[5]))
    return S

In [11]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return D

In [ ]:
# Example
delayer(0,10,'2018-06-04_12:38')

##### Shortcut : séquentiel

In [12]:
def setup() :
    Src_dir = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
    Src_get = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
    Rslt_dir = \
        execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
    Src_dir.run().ok
    Src_get.run().ok
    Rslt_dir.run().ok


In [ ]:

def parl_exec () :
    setup()
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Parallel simulation

In [13]:
# Thread pour la simulation // opt.
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""
    
    # Paramètrage du thread
    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
    
    # Set la valeur du délai
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        
        # make run sur toutes la machine dont le thread s'occupe avec ses paramètres
        execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [14]:
# Test : trajectoire jusqu'à la ième machine raccrochée à la trajectoire initiale
def coherence(tab,i):
     return all(x == True for x in tab[0:i])

In [15]:
# Formate le tableau en string pour pouvoir etre passé en ligne de commande
def format_cmd(data):
    return (' '.join(str(e) for e in data))

In [62]:
# SIMULATION
## Simulation simple en diminuant le nbr de machines 
def simulation(N,NPROCS,mode):
    setup() 
    
    # Décalage initial nul  
    D = [[init_tab(NPROCS,0) for x in range(nodecount)] for y in range(nodecount)] 
    
    # Dates
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    date = date_precise.split('_')[0]
    
    # Tableau de threads
    machines = init_tab(nodecount,None)
    
    # Initialisation du tableau des tests interprocessus (état final i = état initial i+1)
    join = init_tab(nodecount-1,None)
    test_join =  init_tab(nodecount-1,None)
    path = [0]
    for j in range(0,nodecount) :
        print("ROUND "+str(j) +"   ---------------------------------------------------------\n")
        
        # Paramètrage des machines
        for i in range (0,nodecount) : 
            
            # Calcul min et max des itérations pour chaque machine
            MIN = (int)((i*N)/nodecount)
            MAX = (int)(((i+1)*N)/nodecount)
            
            # Calcul des délais après le premier tour
            if (i != 0 and j!=0 ):
                
                # Mise à jour de la matrice des délais
                # D[j][i] = délai sur lequel la machine d'avant a terminé sa trajectoire = delayer (i-1,...)
                D[j][i] = delayer(i-1,NPROCS,date_precise)
                
                
                # D sur lequel la machine d'avant i-1 a terminé sa trajectoire
                delay = format_cmd(D[j][i])  
                
                # D avec lequel la machine i a calculé sa trajectoire en j-1
                previous_D = format_cmd(D[j-1][i])
                #print("Dprime =  : "+ delay+ "         D : "+str(previous_D)+"\n")
                
                # Vérification Dprime et D se rejoignent-ils?
                # Join[i] = True si la machine i et i+1 se rejoignent
                join[i-1] = all(x1 == x2 for x1, x2 in zip(delay, previous_D ))
                
                column = (list(zip(*D))[i])
                test_join[i-1] = any(all(x1 == x2 for x1, x2 in zip(delay,format_cmd(column[k]))) for k in range(j))
      
                

                
                for z in range(len(path),nodecount):
                    for w in range(1,j):
                        if(all(x1 == x2 for x1, x2 in zip(format_cmd(D[path[len(path)-1]+1][z]),format_cmd(column[w])))):
                            path.append(w)
                       # print("heeeey"+str(path[len(path)-1]))
                            #print("Round : "+ str(w)+" Equal:" +format_cmd(D[w][z]) +" and "+format_cmd(column[z]))
                        
                
                """  
                s = [[str(e) for e in row] for row in D]
                lens = [max(map(len, col)) for col in zip(*s)]
                fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
                table = [fmt.format(*row) for row in s]
                print ('\n'.join(table))    """

                    
            # Nouveau délai = D sur lequel la machine d'avant i-1 a terminé sa trajectoire
            # Or D[j][i] = delayer(i-1)
            delay = format_cmd(D[j][i])
            
            # Création des threads
            machines[i] = ParSimer(i,MIN,MAX,N,delay)
            
            # Quand le tableau est entièrement calculé (i = nodecount - 1)
            # Affichage : les trajectoires se rejoignent-elles?  
            if(i == nodecount-1):
                #print("JOIN : " + str(join)+"\n")
                print("path : "+str(path))
                print("test JOIN : " + str(test_join)+"\n")
            # Test cohérence de trajectoire : SIMULTATION TERMINEE     
            if(coherence(join,len(join))):
                print("Complete path computed : END OF SIMULATION\n" + "Rounds taken : " + str (j))
                return;
                          
        # Lancement de la simulation
        for i in range(j,nodecount):
            machines[i].start()
            
        # Fin de simulation
        for i in range(j,nodecount) :
            machines[i].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok  


In [63]:
# Nbr pcs
NPROCS = 3
    
# Nbr itérations
N = 1000

# mode de génération de l'état initial à chaque round
mode = ["FIX","RAND","RAND_INF","RAND_SUP"]

# Lancement de la simulation
simulation(N,NPROCS,mode[1])

ROUND 0   ---------------------------------------------------------

test JOIN : [None, None, None]

parasilo-23 => 0 NPROCS = 3 MIN = 0 MAX = 250 N = 1000 DELAY = '0 0 0'

parasilo-24 => 1 NPROCS = 3 MIN = 250 MAX = 500 N = 1000 DELAY = '0 0 0'

parasilo-25 => 2 NPROCS = 3 MIN = 500 MAX = 750 N = 1000 DELAY = '0 0 0'

parasilo-26 => 3 NPROCS = 3 MIN = 750 MAX = 1000 N = 1000 DELAY = '0 0 0'

ROUND 1   ---------------------------------------------------------

path : [0]
path : [0]
path : [0]
test JOIN : [False, False, False]

parasilo-24 => 1 NPROCS = 3 MIN = 250 MAX = 500 N = 1000 DELAY = '4 0 9'

parasilo-25 => 2 NPROCS = 3 MIN = 500 MAX = 750 N = 1000 DELAY = '3 9 0'

parasilo-26 => 3 NPROCS = 3 MIN = 750 MAX = 1000 N = 1000 DELAY = '3 3 0'

ROUND 2   ---------------------------------------------------------

path : [0, 1]
path : [0, 1]
path : [0, 1]
test JOIN : [True, False, False]

parasilo-25 => 2 NPROCS = 3 MIN = 500 MAX = 750 N = 1000 DELAY = '17 0 19'

parasilo-26 => 3 NPROCS

#### Publication des résultats

In [46]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"

[master c84d994] Automatic results update : Time_Bcast---rennes
 Committer: Najwa Ez Zine <nezzine@frennes>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 21 files changed, 601 insertions(+), 279 deletions(-)
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-07/parasilo-21_2018-06-07_10:55.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-07/parasilo-21_2018-06-07_11:15.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-07/parasilo-21_2018-06-07_11:17.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-07/parasilo-21_201

#### Fin d'expérience : suppression du job

In [ ]:
oardel(jobid)